In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [2]:
source_url = 'https://www.novelupdates.com/series-ranking/?rank=popular&pg=1'
#search "my user agent" in Google paste below
user_agent = {'User-Agent':''}
response = requests.get(source_url,headers=user_agent)
soup = BeautifulSoup(response.content,'html.parser')

In [9]:
all_links = []
for i in range(1,42):
    source_url = f'https://www.novelupdates.com/series-ranking/?rank=popular&pg={i}'
    #search "my user agent" in Google paste below
    user_agent = {'User-Agent':''}
    response = requests.get(source_url,headers=user_agent)
    soup = BeautifulSoup(response.content,'html.parser')
    novels = soup.find_all('div', class_="search_main_box_nu")
    links = [n.a.get("href") for n in novels]
    all_links.extend(links)

In [10]:
novel_review =[]
batch_size = 30
for batch_start in range(0, len(all_links), batch_size):
    batch_links = all_links[batch_start:batch_start + batch_size]


    for l in batch_links:
        try:
            #search "my user agent" in Google paste below
            user_agent = {'User-Agent':''}
            page_number = 1
            while True:
                com = f'comment-page-{page_number}/#comments'
                com_links = f"{l}{com}"
                response = requests.get(com_links,headers=user_agent)
                new_soup = BeautifulSoup(response.content,'html.parser')
                title = new_soup.find('div',class_="seriestitlenu").text
                user_name = new_soup.find_all('a',id="revname")
                user_names = [u.text for u in user_name]
                user_rating_tags = new_soup.find_all('div', class_='w-comments-item')
                user_ratings = [len(ur.find_all('i', class_='fa fa-star')) for ur in user_rating_tags]

                for username, rating in zip(user_names, user_ratings):
                    novel_review.append((title, username, rating))
                
                next_page = new_soup.find('a', class_='next page-numbers')
                if next_page:
                    page_number += 1
                    time.sleep(1)
                else:
                    break
                
            
        except (requests.exceptions.RequestException, AttributeError) as e:
            print(f"An error occurred for {l}: {e}")
            continue
        
df = pd.DataFrame(novel_review, columns=['Title', 'User Name','Rating'])
print(df)


                             Title       User Name  Rating
0      Trash of the Count’s Family        Ztarfish       0
1      Trash of the Count’s Family            Len_       5
2      Trash of the Count’s Family           Nocta       5
3      Trash of the Count’s Family            Jass       2
4      Trash of the Count’s Family     sesamechikn       5
...                            ...             ...     ...
93519    Jiang Hu’s Road is Curved   yourstrulylen       0
93520    Jiang Hu’s Road is Curved  miaomiaomiao28       0
93521    Jiang Hu’s Road is Curved          Kubera       5
93522    Jiang Hu’s Road is Curved         lourdes       5
93523    Jiang Hu’s Road is Curved           Uhoh_       5

[93524 rows x 3 columns]


In [7]:
df.head

<bound method NDFrame.head of                              Title       User Name  Rating
0      Trash of the Count’s Family        Ztarfish       0
1      Trash of the Count’s Family            Len_       5
2      Trash of the Count’s Family           Nocta       5
3      Trash of the Count’s Family            Jass       2
4      Trash of the Count’s Family     sesamechikn       5
...                            ...             ...     ...
13589             I am the Monarch         Renaxan       4
13590             I am the Monarch  KirbyReviews44       5
13591             I am the Monarch  PringleStorian       4
13592             I am the Monarch        LeakLeaf       5
13593             I am the Monarch        donzel00       4

[13594 rows x 3 columns]>

In [11]:
df.to_csv("novel_rating")